# LangChain Academy

## Context

LangChainでは、LLMアプリケーションを簡単に構築できることを目指しています。LLMアプリケーションの一つにエージェントがあります。エージェントを構築することで、これまで不可能であった様々なタスクを自動化することができるため、多くの注目を集めています。

しかし実際には、このようなタスクを確実に実行するシステムを構築するのは非常に困難です。ユーザと協力してエージェントを本番稼動させるにつれ、より多くの制御が必要であることが分かってきました。エージェントが常に特定のツールを最初に呼び出したり、状態に応じて異なるプロンプトを使用する必要があるかもしれません。

この問題に取り組むために、私たちは[LangGraph](https://langchain-ai.github.io/langgraph/) - エージェントとマルチエージェントアプリケーションを構築するためのフレームワークを構築しました。LangChainパッケージとは別に、LangGraphの中心的な設計思想は、開発者がエージェントワークフローに、実世界の複雑なシステムに適した、より良い精度と制御を追加できるようにすることです。

## Course Structure

コースはモジュールで構成されており、各モジュールはLangGraphに関連する特定のテーマに焦点を当てています。各モジュールはノートブックで構成されています。各ノートブックにはビデオが付いていて、コンセプトの説明をしていますが、ノートブックは独立したものです。各モジュールのフォルダには`studio`フォルダがあり、LangGraphアプリケーションを作るためのIDEである[LangGraph Studio](https://github.com/langchain-ai/langgraph-studio)に読み込むことができるグラフが含まれています。

## Setup

始める前に、`README`の指示に従って、環境の構築と依存関係のインストールを行ってください。

## Chat Models

このコースでは[Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models)を使います。このモデルは入力として一連のメッセージを受け取り、出力としてチャットメッセージを返します。LangChainはチャットモデルをホストしていません。[こちら](https://python.langchain.com/v0.2/docs/integrations/chat/) はLangChain内のサードパーティチャットモデル統合のリストです！デフォルトでは、コースは[ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/)を使用します。前述の通り、`OPENAI_API_KEY`があることを確認してください。

`OPENAI_API_KEY`が設定されているか確認してみましょう。


In [13]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [5]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

[こちら](https://python.langchain.com/v0.2/docs/how_to/#chat-models)にチャットモデルでできることについての便利なハウツーがありますが、以下にハイライトをいくつか紹介します。README にあるように `pip install -r requirements.txt` を実行したなら、`langchain-openai` パッケージがインストールされています。これで `ChatOpenAI` モデルオブジェクトをインスタンス化することができます。初めて API にサインアップする場合は、どのモデルにも適用できる [無料クレジット](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) を受け取る必要があります。様々なモデルの価格は [こちら](https://openai.com/api/pricing/) を参照してください。ノートブックのデフォルトは`gpt-4o`で、品質、価格、速度のバランスが良いからです[詳しくはこちら](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini)が、低価格の`gpt-3.5`シリーズを選ぶこともできます。

チャットモデルで設定できる[いくつかの標準パラメータ](https://python.langchain.com/v0.2/docs/concepts/#chat-models)があります。最も一般的なものは2つあります：

* `model`: モデルの名前
* `temperature`: サンプリング温度

`Temperature` はモデルの出力のランダム性や創造性をコントロールします。`Temperature`が0に近いほど正確さや事実に基づいた回答を必要とするタスクに適しています。高温 (1 に近い) は、創造的なタスクや多様な回答を生成するのに適しています。



In [15]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

LangChainのチャットモデルには多くの[デフォルトメソッド](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)があります。ほとんどの場合、以下のメソッドを使います：

* `stream`: レスポンスのチャンクをストリームバックします。
* `invoke`: 入力に対してチェーンを呼び出します。

そして、前述のように、チャットモデルは入力として[messages](https://python.langchain.com/v0.2/docs/concepts/#messages)を受け取ります。メッセージにはロール（誰がメッセージを言っているかを表す）とコンテンツプロパティがあります。これについては後で詳しく説明しますが、ここでは基本的なことだけを説明します。


In [16]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_8080548e81', 'finish_reason': 'stop', 'logprobs': None}, id='run-d3be3fde-22e0-4a80-8a09-dcaac666ab21-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

`AIMessage` のレスポンスが返ってきます。また、文字列でチャットモデルを呼び出すことができることに注意してください。文字列が入力として渡されると、それは `HumanMessage` に変換され、そしてあらかじめ指定したモデルに渡されます。


In [4]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6f6b682-e29a-44de-b45e-79fad1e405e5-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [5]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c75d3f0f-2d71-47be-b14c-42b8dd2b4b08-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

インターフェイスは全てのチャットモデルで一貫しており、モデルは通常、各ノートブックの起動時に一度だけ初期化されます。

そのため、他のプロバイダを強く希望する場合、ダウンストリームのコードを変更することなく、簡単にモデルを切り替えることができます。


## Search Tools

また、READMEには[Tavily](https://tavily.com/)があります。これはLLMとRAGのために最適化された検索エンジンで、効率的で迅速かつ持続的な検索結果を目指しています。前述のように、登録は簡単で、寛大な無料利用枠を提供しています。いくつかのレッスン（module4）ではデフォルトでTavilyを使いますが、もちろん、自分でコードを変更したい場合は他の検索ツールを使うこともできます。


In [6]:
_set_env("TAVILY_API_KEY")

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("LangGraphとはなんですか?")

In [9]:
search_docs


[{'url': 'https://hamaruki.com/langgraph-intro-stateful-multi-agent-apps/',
  'content': 'LangGraphとは. LangGraphは、大規模言語モデル（LLM）を使用して、状態管理が可能なマルチエージェントアプリケーションを構築するためのライブラリです。. このライブラリは、LangChain社によって開発されました。. LangGraphの特徴は、以下の3つのキーワードで ...'},
 {'url': 'https://blog.elcamy.com/posts/136913a3/',
  'content': 'LangGraphとは. LangGraphは、LLMを使って複数のアクターが連携するアプリケーションを構築するためのライブラリです。. エージェントやマルチエージェントのワークフローを作成するために使われます。. 他のLLMフレームワークと比較して、LangGraphは以下の5つの ...'},
 {'url': 'https://qiita.com/ssc-ymuramatsu/items/af4bf120b8be2076b66d',
  'content': 'LangGraph は状態を保持してグラフ上を移動しながら各ノードで処理を行うことでタスクを行います。. Wikipedia にあるようにグラフとは. 数学のグラフ理論におけるグラフ（英: graph）とは数学的構造の一つ。. 対象の集合で、対象の一部が相互に何らかの脈絡で ...'}]